https://www.quantconnect.com/docs/v2/writing-algorithms/trading-and-orders/option-strategies/bull-call-spread

 Introduction

Bull call spread, also known as long call spread, consists of buying an ITM call and selling an OTM call. Both calls have the same underlying Equity and the same expiration date. The OTM call serves as a hedge for the ITM call. The bull call spread profits from a rise in underlying asset price.


In [1]:
# region imports
from AlgorithmImports import *
# endregion

class CrawlingBlackCaribou(QCAlgorithm):

    def initialize(self) -> None:
        self.set_start_date(2024, 1, 1)
        self.set_end_date(2024, 2, 5)
        self.set_cash(500000)

        self.universe_settings.asynchronous = True
        option = self.add_option("SPY", Resolution.MINUTE)
        self._symbol = option.symbol
        option.set_filter(lambda universe: universe.include_weeklys().call_spread(30, 5))


The call_spread filter narrows the universe down to just the two contracts you need to form a bull call spread.

In the on_data method, select the expiration and strikes of the contracts in the strategy legs.

In [2]:
def on_data(self, slice: Slice) -> None:
        if self.portfolio.invested:
            return

        # Get the OptionChain
        chain = slice.option_chains.get(self._symbol, None)
        if not chain:
            self.log(str(self.time)+str(' ')+str('No chain available'))
            return
        
        # Select the call Option contracts with the furthest expiry
        expiry = max([x.expiry for x in chain])
        calls = [i for i in chain if i.expiry == expiry and i.right == OptionRight.CALL]
        if calls == 0:
            self.log(str(self.time)+str(' ')+str(expiry))
            return
        self.log(str(self.time)+str(' ')+ str('len of calls is ')+str(len(calls)))
        # Select the ITM and OTM contract strike prices from the remaining contracts
        strikes = [x.strike for x in calls]
        otm_strike = max(strikes)
        itm_strike = min(strikes)

        option_strategy = OptionStrategies.bull_call_spread(self._symbol, itm_strike, otm_strike, expiry)
        self.buy(option_strategy, 1)

NameError: name 'Slice' is not defined

In [3]:
option_strategy = OptionStrategies.bull_call_spread(self._symbol, itm_strike, otm_strike, expiry)
self.buy(option_strategy, 1)

NameError: name 'OptionStrategies' is not defined

In [4]:
itm_call = [x for x in calls if x.strike == itm_strike][0]
otm_call = [x for x in calls if x.strike == otm_strike][0]

legs = [
    Leg.create(itm_call.symbol, 1),
    Leg.create(otm_call.symbol, -1)
]
self.combo_market_order(legs, 1)

NameError: name 'calls' is not defined